In [28]:
import os
from dotenv import load_dotenv
import dtlpy as dl
from rfdetr.util.coco_classes import COCO_CLASSES



In [29]:
use_rc_env = False
if use_rc_env:
    dl.setenv('rc')
else:
    dl.setenv('prod')
if dl.token_expired():
    dl.login()


In [30]:
if use_rc_env:
    project = dl.projects.get(project_name='HusamDimo')
else:
    project = dl.projects.get(project_name='ShadiDemo')

In [31]:
models = project.models.list().print()



+----+--------------------------+----------------------------------------------------------+----------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [33]:
# Publish your app
dpk = project.dpks.publish()
# Install or update the application
try:
    app = project.apps.get(app_name=dpk.display_name)
    app.dpk_version = dpk.version
    app.update()
except dl.exceptions.NotFound:
    print("installing ...")
    app = project.apps.install(dpk=dpk)

Command Progress: 100%|██████████| 100/100 [00:02<00:00, 34.31it/s]


In [ ]:
model = project.models.get(model_name = 'rf-detr')

In [ ]:
model.open_in_web()

set the lables from COCO_CLASSES

In [7]:
model.labels = list(COCO_CLASSES.values())

In [ ]:
len(COCO_CLASSES.values())


In [ ]:
artifact = model.artifacts.upload(filepath='rf-detr-base-coco.pth')


upload the weights file


In [ ]:
print(f"model status before artifact upload: {model.status}")
# Upload your weights
artifact = model.artifacts.upload(filepath='rf-detr-base-coco.pth')

# Update the configuration
model.configuration['weights_filename'] = artifact.filename


model = project.models.get(model_name = 'rf-detr')
print(f"model status after artifact upload: {model.status}")


model = project.models.get(model_name = 'rf-detr')
print(f"model status after deploy: {model.status}")

In [ ]:
model.status = 'deployed'
model.update()

# # # Deploy your model
# print("-##################\n##################\n##################\n##################\n##################\n deploy only once \n##################\n##################\n##################\n##################\n##################\n")
# model.deploy()

In [ ]:
cloned_model_taxi = project.models.get(model_name='rf-detr-tex4l')
artifact = cloned_model_taxi.artifacts.upload(filepath='model.pth')

# Update the configuration
cloned_model_taxi.configuration['weights_filename'] = artifact.filename

model.update()


In [ ]:
model = project.models.get(model_name = 'rf-detr')

dataset = project.datasets.get(dataset_name='husam_test_ds' if use_rc_env else 'husam_dogs_ds')
image_url = 'https://media.roboflow.com/notebooks/examples/dog-2.jpeg'
item_3 = dataset.items.upload(image_url)
image_url = 'https://media.roboflow.com/notebooks/examples/dog-3.jpeg'
item_4 = dataset.items.upload(image_url)
prediction_3 = model.predict(item_ids=[item_3.id,item_4.id],dataset_id=dataset.id)
prediction_3.wait()
item_3.open_in_web()

In [7]:
item_3.open_in_web()

In [14]:
model = project.models.get(model_name = 'rf-detr')
# model.open_in_web()
# project.datasets.get(dataset_name='husam_train_dataset').open_in_web()

In [16]:

# Train with data splitting and validation
train_filters = dl.Filters(field='dir', values='/train')
val_filters = dl.Filters(field='dir', values='/validation')


model = model.clone(
    model_name='rd-dert-animals-sdk-3',
    description='Husam trained model',
    dataset=project.datasets.get(dataset_name='husam_train_dataset'),
    project_id=project.id,
    train_filter=train_filters,
    validation_filter=val_filters,
)

In [ ]:
exec = model.train()

In [ ]:
s = dl.services.get(service_id="67b4cb48e26f1d1d9699a4c4")
s.log(system=True,
          follow=True)

In [7]:
s = dl.services.get(service_id="680dfb59a8bde1d6431b590f")
s.log(follow=True)

In [26]:
trained_model = project.models.get(model_name = 'rf-detr-clone-2804-matrics')
trained_model.artifacts.list()

Iterate Pages: 100%|██████████| 1/1 [00:00<00:00, 331.28it/s]


[ItemArtifact(dataset_url='https://gate.dataloop.ai/api/v1/datasets/635f9762d2fe493707f03d28', created_at='2025-04-28T14:47:54.102Z', updated_at='2025-04-28T14:47:54.214Z', updated_by='bot.fcf28415-1e5f-4eb1-8cd7-47ec60297290@bot.dataloop.ai', dataset_id='635f9762d2fe493707f03d28', filename='/artifacts/models/rf-detr-clone-2804-matrics/results.json', name='results.json', type='file', id='680f951a3d004e0f03379f9d', spec=None, creator='bot.fcf28415-1e5f-4eb1-8cd7-47ec60297290@bot.dataloop.ai', _description=None, annotations_count=0),
 ItemArtifact(dataset_url='https://gate.dataloop.ai/api/v1/datasets/635f9762d2fe493707f03d28', created_at='2025-04-28T14:47:54.084Z', updated_at='2025-04-28T14:47:54.381Z', updated_by='bot.fcf28415-1e5f-4eb1-8cd7-47ec60297290@bot.dataloop.ai', dataset_id='635f9762d2fe493707f03d28', filename='/artifacts/models/rf-detr-clone-2804-matrics/log.txt', name='log.txt', type='file', id='680f951a88bdef352f14ad37', spec=None, creator='bot.fcf28415-1e5f-4eb1-8cd7-47ec60

In [27]:
trained_model.artifacts.download(artifact_name='metrics_plot.png')

Download Items: 100%|██████████| 1/1 [00:00<00:00,  4.77it/s]


'C:\\Users\\1Husam\\.dataloop\\projects\\ShadiDemo\\datasets\\Binaries\\items\\artifacts/models/rf-detr-clone-2804-matrics/metrics_plot.png'